In [ ]:
#This is the next step from "ADCP_Anomaly_Training.ipynb"

#Now that I have a trained model (trained on BACAX data), lets try to implement it to find drop-outs 
#I am applying this to find drop-outs in BACUS 2 MHZ data

#Kernel:
#adcp_anomaly_env

In [ ]:

#TODO: CONSIDER REVISING THE CODE SO ALL 24hr FILES CLASSIFY AT ONCE!
#   Classifying 1 file at a time is slow!
#Then, Print list of times/beams that have drop-outs, then can run plots seperately after! 
# => DONE!


# Add repo root to Python path - Needed to import from src folder
import sys
from pathlib import Path
repo_root = Path().resolve().parent  # notebooks/ → ADCP-CNN-QAQC
sys.path.append(str(repo_root))

#I used to code above to create a new function and several methods
import os
from src import detect_nortek_dropouts

#For Debugging:
import importlib
importlib.reload(detect_nortek_dropouts)


# Monthly Mat data to convert (directory path, it will find the mat files)
data_parent = r'F:\Documents\Projects\ADCP\scan_for_data\BACUS\ADCP2MHZ\\'

#For one specific folder:
#folder_list = ['20240801']

folder_list = ['20131101', '20141201', '20161101', '20170501', 
                '20220101', '20220201', '20220601', '20220801',
                '20221001', '20230201', '20230701', '20240201',
                '20240801', '20250201']

# folder_list = ['20250701']


#For many folders
#child_folders = os.listdir(data_parent)
#folder_list = child_folders[-12:-1]

#Inputs:
variant = 'resnet50'
#variant = 'TemporalCNN'

if variant == 'TemporalCNN':
    model_path = r"F:\Documents\GitHub\ml_development\ADCP_ML\\" + "best_model_20250508.pt"
else:
    model_path = r"F:\Documents\GitHub\ml_development\ADCP_ML\\" + f"best_model_{variant}.pt"

h5_monthly_folder = r'F:\Documents\Projects\ML\ADCP_ML\BACUS\h5_files\\' # Define output folder -  Monthly h5
h5_24hr_folder = r'F:\Documents\Projects\ML\ADCP_ML\BACUS\h5_24h_files\\'  #Output folder - 24hr h5:

do_delete = 1

#Create a text file for keeping track of detections:
log_path = os.path.join(h5_24hr_folder, f"dropout_detections_{variant}.txt")
# Create or clear the file
with open(log_path, "w") as f:
    f.write("start_time, end_time, class, beam, duration_minutes\n")


for folder in folder_list:
    # Path to your .mat file
    file_list = os.listdir(data_parent + folder)
    mat_files = {k for k in file_list if os.path.splitext(k)[1] == ".mat"}
    print(mat_files)

    #Combine the filenames into a proper path:
    mat_paths = []
    for filename in mat_files:
        mat_paths.append(data_parent + folder + '\\' + filename) 

    #Run the extraction
    for mat_path in mat_paths:
        detect_nortek_dropouts.run_classify(model_path, mat_path, h5_monthly_folder, h5_24hr_folder, log_path, create_plots = 0) 
        #detect_nortek_dropouts.run_classify(model_path, mat_path, h5_monthly_folder, h5_24hr_folder) 

        if do_delete:
            # After detection and plotting, delete all .h5 files
            for h5_folder in {h5_monthly_folder, h5_24hr_folder}:
                file_list = os.listdir(h5_folder)
                h5_files = sorted(k for k in file_list if k.endswith(".h5"))

                for h5_file in h5_files:
                    full_path = os.path.join(h5_folder, h5_file)
                    try:
                        os.remove(full_path)
                        print(f"Deleted {h5_file}")
                    except Exception as e:
                        print(f"Failed to delete {h5_file}: {e}")

In [ ]:

#TODO: CONSIDER REVISING THE CODE SO ALL 24hr FILES CLASSIFY AT ONCE!
#   Classifying 1 file at a time is slow!
#Then, Print list of times/beams that have drop-outs, then can run plots seperately after! 
# => DONE!

#I used to code above to create a new function and several methods
import os
from src import detect_nortek_dropouts

#For Debugging:
import importlib
importlib.reload(detect_nortek_dropouts)


# Monthly Mat data to convert (directory path, it will find the mat files)
data_parent = r'F:\Documents\Projects\ADCP\scan_for_data\BACUS\ADCP2MHZ\\'

#For one specific folder:
#folder_list = ['20240801']

folder_list = ['20131101', '20141201', '20161101', '20170501', 
                '20220101', '20220201', '20220601', '20220801',
                '20221001', '20230201', '20230701', '20240201',
                '20240801', '20250201']

# folder_list = ['20250701']


#For many folders
#child_folders = os.listdir(data_parent)
#folder_list = child_folders[-12:-1]

#Inputs:
variant = 'resnet50'
#variant = 'TemporalCNN'

if variant == 'TemporalCNN':
    model_path = r"F:\Documents\GitHub\ml_development\ADCP_ML\\" + "best_model_20250508.pt"
else:
    model_path = r"F:\Documents\GitHub\ml_development\ADCP_ML\\" + f"best_model_{variant}.pt"

h5_monthly_folder = r'F:\Documents\Projects\ML\ADCP_ML\BACUS\h5_files\\' # Define output folder -  Monthly h5
h5_24hr_folder = r'F:\Documents\Projects\ML\ADCP_ML\BACUS\h5_24h_files\\'  #Output folder - 24hr h5:

do_delete = 0

#Create a text file for keeping track of detections:
log_path = os.path.join(h5_24hr_folder, f"dropout_detections_{variant}.txt")
# Create or clear the file
with open(log_path, "w") as f:
    f.write("start_time, end_time, class, beam, duration_minutes\n")


mat_paths = []
for folder in folder_list:
    # Path to your .mat file
    file_list = os.listdir(data_parent + folder)
    mat_files = {k for k in file_list if os.path.splitext(k)[1] == ".mat"}
    print(mat_files)

    #Combine the filenames into a proper path:
    
    for filename in mat_files:
        mat_paths.append(data_parent + folder + '\\' + filename) 

#Run the extraction
for mat_path in mat_paths:
    detect_nortek_dropouts.run_classify(model_path, mat_path, h5_monthly_folder, h5_24hr_folder, log_path, create_plots = 0) 
    #detect_nortek_dropouts.run_classify(model_path, mat_path, h5_monthly_folder, h5_24hr_folder) 

    if do_delete:
        # After detection and plotting, delete all .h5 files
        for h5_folder in {h5_monthly_folder, h5_24hr_folder}:
            file_list = os.listdir(h5_folder)
            h5_files = sorted(k for k in file_list if k.endswith(".h5"))

            for h5_file in h5_files:
                full_path = os.path.join(h5_folder, h5_file)
                try:
                    os.remove(full_path)
                    print(f"Deleted {h5_file}")
                except Exception as e:
                    print(f"Failed to delete {h5_file}: {e}")

In [9]:
os.getcwd()

'/lustre10/scratch/slonimer'

In [13]:

#Apply on Rorqual to pre-made h5 files

#TODO: CONSIDER REVISING THE CODE SO ALL 24hr FILES CLASSIFY AT ONCE!
#   Classifying 1 file at a time is slow!
#Then, Print list of times/beams that have drop-outs, then can run plots seperately after! 
# => DONE!

#I used to code above to create a new function and several methods
import os
from src import detect_nortek_dropouts

#For Debugging:
import importlib
importlib.reload(detect_nortek_dropouts)

#Inputs:
variant = 'resnet34'
#variant = 'TemporalCNN'

model_path = "best_model_4_resnet34_bs16.pt"

#Already converted to daily h5, so can set these to none
mat_path = None
h5_monthly_folder = None

#Folder with 24hr h5 files
h5_24hr_folder = '/lustre10/scratch/slonimer/BACUS_24hr_h5_test/'


#Create a text file for keeping track of detections:
log_path = os.path.join(h5_24hr_folder, f"dropout_detections_{variant}.txt")
# Create or clear the file
with open(log_path, "w") as f:
    f.write("start_time, end_time, class, beam, duration_minutes\n")


#Run the extraction
detect_nortek_dropouts.run_classify(model_path, mat_path, h5_monthly_folder, h5_24hr_folder, log_path, create_plots = 0) 
#detect_nortek_dropouts.run_classify(model_path, mat_path, h5_monthly_folder, h5_24hr_folder) 

    

📌 Initializing ResNet model (resnet34)

=== Starting Evaluation on Test Set ===
Running drop-out detection on all 24 hr files


In [ ]:
#Can I apply this to the anomalous examples used for training?

import os
from src import detect_nortek_dropouts

#For Debugging:
import importlib
importlib.reload(detect_nortek_dropouts)

#For ALL anomalous files, plot the annotations and the prediction

#Initialize the classification model
model_path = r"F:\Documents\GitHub\ml_development\ADCP_ML\\" + "best_model_20250508.pt"
model = detect_nortek_dropouts.init_model(model_path)

#Define anomaly file paths
file_path = r"F:\Documents\Projects\ML\ADCP_ML\h5_24h_files\\"

#Create a text file for keeping track of detections:
log_path = os.path.join(file_path, "dropout_detections.txt")
# Create or clear the file
with open(log_path, "w") as f:
    f.write("start_time, end_time, class, beam, duration_minutes\n")
    
# Extract anomaly filenames from a text file
with open(r"F:\Documents\Projects\ML\ADCP_ML\\" + "annotated_files.txt", "r") as f:
    #anomaly_files = set(line.strip() for line in f if line.strip())
    anomaly_files = list(dict.fromkeys(line.strip() for line in f if line.strip()))

#Run the classification and plots:
detect_nortek_dropouts.classify_and_plot(model, anomaly_files, file_path)
#detect_nortek_dropouts.classify_and_plot(model, h5_files, h5_24hr_folder)


In [8]:
print(anomaly_files)

['20130619T000000_20130619T235959.h5', '20130620T000000_20130620T235959.h5', '20140829T000000_20140829T235959.h5', '20140830T000000_20140830T235959.h5', '20170830T235731_20170831T235730.h5', '20170901T000230_20170902T000229.h5', '20171118T000000_20171118T235959.h5', '20171119T000000_20171119T235959.h5', '20171120T000000_20171120T235959.h5', '20190511T000000_20190511T235959.h5', '20190512T000000_20190512T235959.h5', '20230624T000000_20230624T235959.h5', '20230625T000000_20230625T235959.h5', '20230701T000230_20230702T000229.h5', '20230702T000000_20230702T235959.h5', '20230703T000000_20230703T235959.h5', '20231114T000000_20231114T235959.h5', '20231115T000000_20231115T235959.h5', '20231116T000000_20231116T235959.h5', '20231117T000000_20231117T235959.h5', '20231118T000000_20231118T235959.h5', '20240406T000000_20240406T235959.h5', '20240625T000000_20240625T235959.h5', '20240626T000000_20240626T235959.h5', '20240627T000000_20240627T235959.h5', '20240628T000000_20240628T235959.h5', '20240629T0